# 要約 
このJupyter Notebookは、LMSYS - Chatbot Arenaコンペティションで使用される、チャットボットの応答の好みを予測するための機械学習モデルの訓練に関するものです。具体的には、Llama3という言語モデルを用いて、ユーザーが好む応答を予測し、その性能を評価しています。

### 問題
このノートブックでは、Chatbot Arenaデータセットに基づいて、どちらのチャットボット応答が好まれるかを予測するモデルを構築しています。ユーザーからのフィードバックに基づいて、異なるモデルの応答の優劣を判断する問題に取り組みます。

### 手法とライブラリ
- **モデル:** Llama3（8ビット精度）をベースとして使用し、LoRA (Low-Rank Adaptation)を導入したカスタムモデル `Llama3ForSFT`を作成しています。これにより、モデルの重量を軽量化し、効率的にトレーニングを行えるようにしています。
  
- **トークナイザー:** `transformers` ライブラリの `AutoTokenizer`を使用して、テキストデータをトークン化しています。

- **データ処理:** `pandas`を使用してデータを読み込み、一部のデータを利用してモデルを訓練しています。また、`datasets` ライブラリを用いてデータセットを管理しています。

- **評価指標:** モデルの性能評価には、対数損失 (`log_loss`) と精度 (`accuracy_score`) が使用されています。`scikit-learn`の機能を活用して計算しています。

### 結果
ノートブック内では、評価用データセットのログ損失が 0.9231 で、リーダーボードのスコアが0.936と示されており、モデルの性能が確認されています。また、訓練時のバッチサイズやエポック数、評価手法についても詳細に説明がなされ、特にGPU環境での実行を考慮した条件が記載されています。

全体として、このノートブックは、ユーザーの応答の好みを予測するための機械学習モデルの実装と、そのトレーニングプロセスを包括的に示しています。

---


# 用語概説 
以下はノートブックで使われているが、初心者には馴染みが薄いかもしれない専門用語の簡単な解説です。

1. **ログ損失 (Log Loss)**:
   - 予測モデルの性能を評価するための指標。確率的な予測が実際のクラスラベルとどれだけズレているかを測定します。小さい値が良いとされ、予測が実際の正解に近いほど小さくなります。

2. **トークナイザー (Tokenizer)**:
   - 自然言語処理で使用するテキストを、機械が理解できる数値データ（トークン）に変換するためのツール。トークンは単語やサブワードのような単位で、これによってモデルがテキストを処理します。

3. **アテンションマスク (Attention Mask)**:
   - アテンション機構を実装する際に使用するマスクで、どの部分が計算の対象であるか（1）または無視すべき部分（0）を示します。従って、パディングされたデータの影響を排除します。

4. **PEFT (Parameter-Efficient Fine-Tuning)**:
   - 限られたパラメータのみを調整してモデルを微調整する手法。これにより、訓練に必要なリソースを削減しつつ、モデルのパフォーマンスを向上させます。このノートブックでは、LoRA（Low-Rank Adaptation）という手法が利用されています。

5. **LoRA (Low-Rank Adaptation)**:
   - 大規模モデルの微調整において、全体の重みを更新するのではなく、低ランクの行列を使用して効率的にモデルを調整する方法。トレーニングの効率を向上させ、必要な計算資源を削減します。

6. **データコレーター (Data Collator)**:
   - バッチ処理の際にデータをまとめるためのツール。特に異なる長さのシーケンスを適切にパディングするなど、データを連結してバッチにする役割を担います。

7. **Gradient Accumulation (勾配蓄積)**:
   - 小さなバッチサイズでモデルをトレーニングする際に、複数のバッチの勾配を蓄積してから更新を行う技術。これにより、実際のバッチサイズを大きくすることができます。

8. **評価戦略 (Evaluation Strategy)**:
   - モデルのトレーニング中に評価を行うタイミングや方法を定義する設定。例えば、エポックごとの評価や特定のステップごとに行うなどがあります。

9. **half-precision (fp16)**:
   - 計算の精度の一つで、16ビットの浮動小数点数を使用すること。これによりメモリ使用量が削減され、計算速度が向上することがありますが、数値の精度は低下することがあります。

10. **タスクタイプ (Task Type)**:
    - モデルが実行する特定のタスクの種類で、ここでは因果推論（Causal Language Modeling）の設定がされている。これは、次に来るトークンを予測するタスクです。

---


## 結果
- [推論コード](https://www.kaggle.com/code/shelterw/sft-llama-3-8b-inference)    

- [ベースモデル: llama-3-8b-Instruct-bnb-4bit](https://huggingface.co/unsloth/llama-3-8b-Instruct-bnb-4bit)

| サブセット | ログ損失 |
| - | - |
| 評価 | 0.9231 |
| LB | 0.936 |

## 注意
コードを再現したい場合は、以下の点に注意してください：
- すべてのデータを使用すること
- per_device_train_batch_size=4に設定すること
- A10を使用して1エポックは約15時間かかること



In [ ]:
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft  # transformers, bitsandbytes, accelerate, peftのパッケージを最新バージョンでインストールします

In [ ]:
import os  # OS関連の機能をインポート
import copy  # オブジェクトをコピーするためのモジュールをインポート
from dataclasses import dataclass  # データクラスを作成するためのモジュールをインポート

import torch  # PyTorchのライブラリをインポート
import torch.nn as nn  # PyTorchのニューラルネットワークモジュールをインポート
import torch.nn.functional as F  # PyTorchの関数型ニューラルネットワークモジュールをインポート
import pandas as pd  # データ処理のためのパンダスライブラリをインポート
import numpy as np  # 数値計算のためのNumPyライブラリをインポート
from datasets import Dataset  # データセット管理のためのライブラリをインポート
from scipy.special import softmax  # softmax関数をインポート
from sklearn.preprocessing import LabelEncoder  # ラベルエンコーディングのためのモジュールをインポート
from transformers import (  # Transformersライブラリから以下のクラスをインポート
    BitsAndBytesConfig,
    LlamaPreTrainedModel,
    LlamaModel,
    AutoTokenizer,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
)
from transformers.modeling_outputs import CausalLMOutputWithPast  # モデルの出力のデータ型をインポート
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType  # PEFTのための設定やモデルをインポート
from sklearn.metrics import log_loss, accuracy_score  # 精度とログ損失の評価指標をインポート

### 設定


In [ ]:
TRAIN_CSV = "/kaggle/input/lmsys-chatbot-arena/train.csv"  # 訓練データのCSVファイルのパスを指定
model_path = "unsloth/llama-3-8b-Instruct-bnb-4bit"  # モデルのパスを指定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPUが利用可能な場合はGPUを使用、そうでなければCPUを使用
MAX_LENGTH = 1024  # トークンの最大長を設定
target_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']  # ターゲットとなるカラムをリスト化
columns_to_vectorize = ["prompt", "response_a", "response_b"]  # ベクトル化するカラムをリスト化

train = pd.read_csv(TRAIN_CSV)  # 訓練データをCSVファイルから読み込み
train = train.head(100)  # 最初の100行を取得
train['label'] = train[target_columns].idxmax(axis=1)  # 各行で最も大きいインデックスをラベルとして設定
label_encoder = LabelEncoder()  # ラベルエンコーダのインスタンスを作成
train['label'] = label_encoder.fit_transform(train['label'])  # ラベルをエンコード
train = train[columns_to_vectorize + ['label']]  # 必要なカラムのみを保持

### トークナイザーとデータセットの準備、評価指標


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)  # 事前学習済みモデルのトークナイザーを取得
tokenizer.add_eos_token = True  # EOSトークンを追加
tokenizer.padding_side = 'right'  # パディングは右側に設定

# ラベルのトークンIDを取得
LABEL_IDS = [tokenizer(i, add_special_tokens=False)["input_ids"][0] for i in ['a', 'b', 'tie']]

def tokenize(example, tokenizer):  # トークン化のための関数を定義
    # プロンプト、応答A、応答Bをそれぞれトークン化
    prompt = tokenizer('<prompt>: ' + " ".join(eval(example['prompt'], {"null": ""})), add_special_tokens=False)["input_ids"]
    response_a = tokenizer('\n\n<response_a>: ' + " ".join(eval(example['response_a'], {"null": ""})), add_special_tokens=False)["input_ids"]
    response_b = tokenizer('\n\n<response_b>: ' + " ".join(eval(example['response_b'], {"null": ""})), add_special_tokens=False)["input_ids"]
    # 最大長を超える場合は部分的にトークン化
    if len(prompt + response_a + response_b) > MAX_LENGTH:
        prompt = tokenizer('<prompt>: ' + eval(example['prompt'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:256]
        response_a = tokenizer('\n\n<response_a>: ' + eval(example['response_a'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:512]
        response_b = tokenizer('\n\n<response_b>: ' + eval(example['response_b'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:512]
    extra_prompt = tokenizer('\n\n---------\nこのプロンプトに対してどちらの応答が良いか？ a または b または tie か？\n\n答え: ', add_special_tokens=False)["input_ids"]

    label_token_id = LABEL_IDS[int(example['label'])]  # 正解ラベルのトークンIDを取得
    input_ids = [tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt + [label_token_id] + [tokenizer.eos_token_id]
    attention_mask = len(input_ids) * [1]  # アテンションマスクを作成
    labels = [-100] * len([tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt) + [label_token_id] + [tokenizer.eos_token_id]  # ラベルを準備
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
def load_data(df, tokenizer):  # データをロードする関数を定義
    raw_datasets = Dataset.from_pandas(df)  # pandas DataFrameからデータセットを作成
    tokenized_datasets = raw_datasets.map(
        tokenize,  # トークン化関数をマップ
        remove_columns=raw_datasets.column_names,  # 使用しないカラムは削除
        fn_kwargs={'tokenizer': tokenizer}  # トークナイザーを引数として渡す
    )
    return tokenized_datasets  # トークナイズされたデータセットを返す

def compute_metrics(pred):  # 精度や損失を計算する関数を定義
    logits, labels = pred  # 予測値とラベルを取得
    preds = logits.argmax(axis=-1)  # 最大のロジットからクラスを予測
    label_tokens_ids = np.array(LABEL_IDS)  # ラベルトークンのIDをNumPy配列に変換
    index_mapping = {value.item(): idx for idx, value in enumerate(label_tokens_ids)}  # インデックスマッピングを作成
    labels = labels[np.isin(labels, label_tokens_ids)]  # ラベルをフィルタリング
    labels = np.array([index_mapping[label.item()] for label in labels])  # ラベルをインデックスに変換
    acc = accuracy_score(labels, preds)  # 精度を計算
    probs = softmax(logits, axis=-1)  # ソフトマックスで確率を計算
    log_loss_ = log_loss(labels, probs)  # ログ損失を計算
    return {'accuracy': acc, 'log_loss': log_loss_}  # 精度とログ損失を辞書で返す

n_splits = 5  # データの分割数を指定
fold_idx = 0  # 現在のフォールドインデックスを初期化
ds = load_data(train, tokenizer)  # データをロード
# n分割交差検証のためのインデックスを作成
folds = [
    (
        [i for i in range(len(ds)) if i % n_splits != fold_idx],
        [i for i in range(len(ds)) if i % n_splits == fold_idx]
    ) 
    for fold_idx in range(n_splits)
]
train_idx, eval_idx = folds[fold_idx]  # トレーニングと評価のインデックスを取得

### モデル


In [ ]:
class Llama3ForSFT(LlamaPreTrainedModel):  # LlamaPreTrainedModelを継承したクラスを定義
    _tied_weights_keys = ["lm_head.weight"]  # 結びつけられた重みのキーを指定
    def __init__(self, config):
        super().__init__(config)  # 親クラスの初期化
        self.model = LlamaModel(config)  # Llamaモデルのインスタンスを作成
        self.vocab_size = config.vocab_size  # 語彙サイズを設定
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)  # 出力層を定義
        self.post_init()  # 初期化処理

    def forward(  # フォワード関数を定義
        self,
        input_ids=None,  # 入力ID
        attention_mask=None,  # アテンションマスク
        position_ids=None,  # 位置ID
        past_key_values=None,  # 過去のキーと値
        inputs_embeds=None,  # 入力の埋め込みベクトル
        labels=None,  # ラベル
        use_cache=None,  # キャッシュを使用するか
        output_attentions=None,  # アテンションを出力するか
        output_hidden_states=None,  # 隠れ状態を出力するか
        return_dict=None,  # 辞書形式で返すか
        cache_position=None,  # キャッシュの位置
    ):
        outputs = self.model(  # モデルを通して出力を取得
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            cache_position=cache_position,
        )
        hidden_states = outputs[0]  # 隠れ状態を取得
        if self.config.pretraining_tp > 1:  # モデル並列化が必要な場合
            lm_head_slices = self.lm_head.weight.split(self.vocab_size // self.config.pretraining_tp, dim=0)  # 重みを分割
            logits = [F.linear(hidden_states, lm_head_slices[i]) for i in range(self.config.pretraining_tp)]  # 各スライスに対するロジットを計算
            logits = torch.cat(logits, dim=-1)  # ロジットを結合
        else:
            logits = self.lm_head(hidden_states)  # 隠れ状態を基にロジットを計算
        logits = logits.float()  # ロジットをfloat型に変換

        loss = None  # 損失を初期化
        if labels is not None:  # ラベルが与えられている場合
            # トークンをシフトして予測を行う
            shift_logits = logits[..., :-1, :].contiguous()  # 最後のトークンを除外
            shift_labels = labels[..., 1:].contiguous()  # 最初のトークンを除外
            # トークンをフラット化
            loss_fct = nn.CrossEntropyLoss()  # クロスエントロピー損失関数を定義
            shift_logits = shift_logits.view(-1, self.config.vocab_size)  # ロジットをフラットに変換
            shift_labels = shift_labels.view(-1)  # ラベルをフラットに変換
            # モデル並列化を有効化
            shift_labels = shift_labels.to(shift_logits.device)  # デバイスを統一

            label_tokens_ids = torch.tensor(LABEL_IDS, device=shift_labels.device)  # ラベルトークンIDをテンソルに変換
            index_mapping = {value.item(): idx for idx, value in enumerate(label_tokens_ids)}  # インデックスマッピングを作成
            true_labels = shift_labels[torch.isin(shift_labels, label_tokens_ids)]  # 正しいラベルをフィルタリング
            true_labels = torch.tensor([index_mapping[label.item()] for label in true_labels], device=true_labels.device)  # インデックスに変更
            true_logits = shift_logits[torch.isin(shift_labels, label_tokens_ids)][:, label_tokens_ids]  # 正しいロジットを取得
            loss = loss_fct(true_logits, true_labels)  # 損失を計算

        return CausalLMOutputWithPast(  # 出力を返す
            loss=loss,  # 損失
            logits=true_logits,  # 正しいロジット
        )

In [ ]:
peft_config = LoraConfig(  # PEFT構成を定義
    r=16,  # ランク
    lora_alpha=32,  # LoRAアルファ
    lora_dropout=0.05,  # LoRAドロップアウト率
    bias='none',  # バイアス
    inference_mode=False,  # 推論モード
    task_type=TaskType.CAUSAL_LM,  # タスクタイプ
    target_modules=['q_proj', 'k_proj', 'v_proj'],  # 対象モジュールを指定
)

model = Llama3ForSFT.from_pretrained(  # 事前学習済みのモデルをロード
    model_path, 
    torch_dtype=torch.float16,  # 半精度でロード
)
model.config.use_cache = False  # キャッシュを使用しない設定
model = prepare_model_for_kbit_training(model)  # kビットトレーニング用にモデルを準備
model = get_peft_model(model, peft_config)  # PEFTモデルを取得
print(model)  # モデルの概要を表示
model.print_trainable_parameters()  # 訓練可能なパラメータを表示

#### 訓練引数


In [ ]:
args = TrainingArguments(  # 訓練引数を定義
    output_dir='output',  # 出力ディレクトリ
    overwrite_output_dir=True,  # 出力ディレクトリを上書きするか
    evaluation_strategy="epoch",  # 評価の戦略
    save_strategy="steps",  # 保存の戦略
    save_steps=200,  # 200ステップごとに保存
    save_total_limit=1,  # 最大保存数
    logging_strategy="steps",  # ロギングの戦略
    logging_steps=10,  # 10ステップごとにログ
    warmup_steps=20,  # ウォームアップステップ数
    optim="adamw_8bit",  # 最適化アルゴリズム
    learning_rate=2e-4,  # 学習率
    per_device_train_batch_size=2,  # デバイスごとのトレーニングバッチサイズ
    per_device_eval_batch_size=4,  # デバイスごとの評価バッチサイズ
    gradient_accumulation_steps=2,  # 勾配蓄積ステップ数
    num_train_epochs=1,  # 訓練エポック数
    fp16=True,  # フロート16を使用
    metric_for_best_model="log_loss",  # 最良モデルの評価指標
    greater_is_better=False,  # 指標が大きいほど良いか
    report_to="none",  # どこに報告するか
)

### 訓練開始！


In [ ]:
trainer = Trainer(  # トレーナーインスタンスを作成
    args=args,  # 訓練引数
    model=model,  # 使用するモデル
    train_dataset=ds.select(train_idx),  # トレーニングデータセット
    eval_dataset=ds.select(eval_idx),  # 評価データセット
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),  # データコレーター
    compute_metrics=compute_metrics,  # 評価指標計算用関数
)
trainer.train()  # 訓練を開始

---

# コメント 

> ## KeShuang Liu
> 
> こんにちは、トレーニングにmax_lengthが1024、推論に2400と設定されている理由を知りたいです。トレーニングでmax_length=2400を使用したことはありますか？
> 

---

> ## OHIRA
> 
>素晴らしい作品をありがとうございます！
> 
> コードについて質問があります。
> 
> load_best_model_at_end = Trueを設定した場合、
> 
> args = TrainingArguments(
>     output_dir='/kaggle/output',
>     overwrite_output_dir=True,
>     evaluation_strategy="steps",
>     save_strategy="steps",
>     save_steps=20,
>     save_total_limit=5,
>     logging_strategy="steps",
>     logging_steps=20,
>     warmup_steps=20,
>     optim="adamw_8bit",
>     learning_rate=2e-4,
>     per_device_train_batch_size=2,
>     per_device_eval_batch_size=4,
>     gradient_accumulation_steps=2,
>     num_train_epochs=1,
>     fp16=True,
>     metric_for_best_model="log_loss",
>     greater_is_better=False,
>     report_to="none",
>     load_best_model_at_end=True
> )
> 
> 評価セットで5つの最良モデルのパラメータを取得できますか？
> 
> それとも最後の5つのモデルのパラメータを取得するのですか？
> 
> 

---

> ## daichisaito-cs
> 
> 素晴らしい作品を共有していただきありがとうございます！
> 
> 質問があります：
> 
> Evalのスコア0.9231、LBのスコア0.936を再現するには何エポック必要ですか？
> 
> デフォルトの訓練エポック数は1に設定されていますが、これがこれらのスコアを得るために使用された値と同じですか？
> 
> ありがとう
> 
> 
> > ## ShelterWTopic 著者
> > 
> はい。       
> > 
> 

---

> ## Lorry Zou
> 
> Gemma2 9Bをこの方法（次の単語予測）で訓練しようとしましたか？Llama3の場合、この方法はLlamaForSequenceClassificationを直接使用するよりもはるかに良いパフォーマンスを持つようです。
> 

---

> ## Stringersolo
> 
> こんにちは [@shelterw](https://www.kaggle.com/shelterw)、共有していただきありがとうございます。同じ結果を再現する際に問題があります。具体的には、トレーニング中は指標がほぼ同じですが、LBでスコアを計算すると約1.2で、非常に奇妙で平均的な/ランダムな予測に近いです。
> 
> モデルの重みを直接読み取ろうとしましたが、役に立ちませんでした：
> 
> model_0.load_state_dict(torch.load(RAW_WEIGHTS), strict=False)
> 
> 
> > ## ShelterWTopic 著者
> > 
> モデルをロードする際にlm headの重みがランダムに再ロードされることが原因かもしれません。
> > 
> transformersとpeftのバージョンを更新するか、LlamaPretrainedModelの代わりにLlamaCausalModelクラスを継承するようにしてください。
> > 
> Gemma2を使用したときにも同じ現象が起きましたが、奇妙です。
> > 
> [こちら](https://www.kaggle.com/competitions/lmsys-chatbot-arena/discussion/518408#2912471)を参照してください。
> > 
> 
> > > ## Stringersolo
> > > 
> > > ありがとう[@shelterw](https://www.kaggle.com/shelterw)、試してみます。このリンクでスコアレイヤーを保存するように提案されました：
> > > 
> > > torch.save(classifier.score.state_dict(), f'{output_directory_path}/score_state_dict.pth')
> > > 
> > > あなたの場合は次のようになります：
> > > 
> > > torch.save(trainer.model.lm_head.state_dict(), f'output/lm_head_dict.pth')
> > > 
> > > ですよね？
> > > 
> > > 
> > > 

---

> ## Yi-Fu Chen
> 
> なぜ.autoModelForCausalLMを直接使用するのではなく、Llama3ForSFTを実装する必要があるのか教えていただけますか？特別な理由はありますか？
> 
> 
> > ## ShelterWTopic 著者
> > 
> > ラベルトークンのロジットと損失のためです。
> > 
> 

---

> ## Eido Mike
> 
> 優れた作品です！共有していただきありがとうございます。
> 
> 

---

> ## AbaoJiang
> 
> こんにちは [@shelterw](https://www.kaggle.com/shelterw)、
> 
> 共有していただきありがとうございます。トレーニングにCAUSAL_LMタスクを使用したことに気付きました。LlamaForSequenceClassificationを使用してトレーニングした場合のパフォーマンスと比較しましたか？
> 
> 
> > ## ShelterWTopic 著者
> > 
> > llama3-8bとSEQ_CLSを比較したことはありませんが、以前の実験ではllama3-8bの方が悪かったですが、gemma2-9bのSEQ_CLSよりは良かったです。
> > 
> 

---

> ## __ChrisQ__
> 
> こんにちは、このノートブックをありがとうございます。
> 
> 一つ質問があります：すべてのデータを使用する場合、評価スコアはどのように計算しますか？
> 
> 
> > ## ShelterWTopic 著者
> > 
> > スクリプトの'compute_metrics'関数は、1エポック後に自動的に計算されます。
> > 
> > 
> > 
> > > ## raconion
> > > 
> > > 'compute_metrics'関数はトレーニングデータの20%をクロスバリデーションに使用します。評価データを使用してモデルをさらにトレーニングしたのでしょうか？「すべてのデータを使用する」とはどういう意味ですか？5分割CVのためにすべてのデータを使用するという意味でしょうか、それともモデルをすべてのデータでトレーニングしますか？
> > > 
> > > 
> > > > ## ShelterWTopic 著者
> > > > 
> > > > トレーニングの80%を使用し、評価の20%を使用します。
> > > 
> > > 
> > > ## raconion
> > > 
> > > ご確認いただきありがとうございます :)
> > > 
> > > 
> > > 
